In [1]:
import os
#! pip install -r requirements.txt
import numpy as np
if np.__version__ != "1.16.0": # 1st time
    !pip install opencv-python numpy==1.16.0 tqdm yacs json_tricks tensorboardX pycocotools scipy 
    exit(0) # restart session to use newly installed numpy

In [2]:
import os
!git clone https://github.com/HRNet/DEKR
!git clone https://github.com/cocodataset/cocoapi.git
!git clone https://github.com/Jeff-sjtu/CrowdPose

Cloning into 'DEKR'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 65 (delta 14), reused 62 (delta 14), pack-reused 0
Unpacking objects: 100% (65/65), done.
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (576/576), done.
Cloning into 'CrowdPose'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 80 (delta 1), reused 4 (delta 1), pack-reused 71
Unpacking objects: 100% (80/80), done.


In [3]:
from google.colab import drive
import shutil
import os
drive.mount('/content/drive/', force_remount=True)
drive_model_file = "/content/drive/My Drive/DEKR/models/pose_coco/pose_dekr_hrnetw32_coco.pth"
model_file = "/content/DEKR/models/pose_coco/pose_dekr_hrnetw32_coco.pth"

if not os.path.exists(model_file):
    os.makedirs("/content/DEKR/models/pose_coco", exist_ok=True)
    shutil.copyfile(drive_model_file, model_file)

Mounted at /content/drive/


In [4]:
import torch
print(torch.__version__)

1.9.0+cu102


In [5]:
import os
os.chdir("/content/cocoapi/PythonAPI")
!make install

# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/loc

In [6]:
import os
os.chdir("/content/CrowdPose/crowdpose-api/PythonAPI")
!git checkout 785e70d269a554b2ba29daf137354103221f479e
!python3 setup.py install

Note: checking out '785e70d269a554b2ba29daf137354103221f479e'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 785e70d fix a bug in AP
running install
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/crowdposetools
copying crowdposetools/cocoeval.py -> build/lib.linux-x86_64-3.7/crowdposetools
copying crowdposetools/__init__.py -> build/lib.linux-x86_64-3.7/crowdposetools
copying crowdposetools/coco.py -> build/lib.linux-x86_64-3.7/crowdposetools
copying crowdposetools/mask.py -> build/lib.linux-x86_64-3.7/crowdposetools
running build_ext
building 'crowd

In [7]:
os.chdir("/content/DEKR")
os.makedirs("data/coco/images", exist_ok=True)
os.chdir("data/coco/images")
#!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!md5sum val2017.zip

os.chdir("/content/DEKR/data/coco/")
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!md5sum annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

--2021-08-20 05:14:02--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.169.25
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.169.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  35.2MB/s    in 23s     

2021-08-20 05:14:25 (34.3 MB/s) - ‘val2017.zip’ saved [815585330/815585330]

442b8da7639aecaf257c1dceb8ba8c80  val2017.zip
--2021-08-20 05:14:27--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.49.188
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.49.188|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[================

In [8]:
os.chdir("/content/DEKR")
!md5sum /content/DEKR/models/pose_coco/pose_dekr_hrnetw32_coco.pth

005dd53616db101483737f9342305b73  /content/DEKR/models/pose_coco/pose_dekr_hrnetw32_coco.pth


In [9]:
!python tools/valid.py \
    --cfg experiments/coco/w32/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140.yaml \
    TEST.MODEL_FILE models/pose_coco/pose_dekr_hrnetw32_coco.pth

=> creating output
=> creating output/coco_kpt/hrnet_dekr/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140
=> creating log/coco_kpt/hrnet_dekr/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140_2021-08-20-05-16
Namespace(cfg='experiments/coco/w32/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140.yaml', opts=['TEST.MODEL_FILE', 'models/pose_coco/pose_dekr_hrnetw32_coco.pth'])
AUTO_RESUME: True
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  BG_WEIGHT: 0.1
  CENTER_SIGMA: 4.0
  DATASET: coco_kpt
  DATASET_TEST: coco
  DATA_FORMAT: zip
  FLIP: 0.5
  INPUT_SIZE: 512
  MAX_NUM_PEOPLE: 30
  MAX_ROTATION: 30
  MAX_SCALE: 1.5
  MAX_TRANSLATE: 40
  MIN_SCALE: 0.75
  NUM_JOINTS: 17
  OFFSET_RADIUS: 4
  OUTPUT_SIZE: 128
  ROOT: data/coco
  SCALE_TYPE: short
  SIGMA: 2.0
  TEST: val2017
  TRAIN: train2017
DATA_DIR: 
DIST_BACKEND: nccl
GPUS: (0,)
LOG_DIR: log
LOSS:
  HEATMAPS_LOSS_FACTOR: 1.0
  OFFSETS_LOSS_FACTOR: 0.03
  WITH_HEATMAPS_LOSS: True
  WITH_OFFSETS_LOSS: True
MODEL:
  INIT_WEI

In [10]:
model_file = "model/rescore/final_rescore_coco_kpt.pth"
if not os.path.exists(model_file):
    os.makedirs("model/rescore", exist_ok=True)
    os.symlink("../../models/pose_coco/pose_dekr_hrnetw32_coco.pth", "model/rescore/final_rescore_coco_kpt.pth")

!python tools/valid.py \
    --cfg experiments/coco/w32/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140.yaml \
    TEST.MODEL_FILE models/pose_coco/pose_dekr_hrnetw32_coco.pth

=> creating output/coco_kpt/hrnet_dekr/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140
=> creating log/coco_kpt/hrnet_dekr/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140_2021-08-20-05-17
Namespace(cfg='experiments/coco/w32/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140.yaml', opts=['TEST.MODEL_FILE', 'models/pose_coco/pose_dekr_hrnetw32_coco.pth'])
AUTO_RESUME: True
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  BG_WEIGHT: 0.1
  CENTER_SIGMA: 4.0
  DATASET: coco_kpt
  DATASET_TEST: coco
  DATA_FORMAT: zip
  FLIP: 0.5
  INPUT_SIZE: 512
  MAX_NUM_PEOPLE: 30
  MAX_ROTATION: 30
  MAX_SCALE: 1.5
  MAX_TRANSLATE: 40
  MIN_SCALE: 0.75
  NUM_JOINTS: 17
  OFFSET_RADIUS: 4
  OUTPUT_SIZE: 128
  ROOT: data/coco
  SCALE_TYPE: short
  SIGMA: 2.0
  TEST: val2017
  TRAIN: train2017
DATA_DIR: 
DIST_BACKEND: nccl
GPUS: (0,)
LOG_DIR: log
LOSS:
  HEATMAPS_LOSS_FACTOR: 1.0
  OFFSETS_LOSS_FACTOR: 0.03
  WITH_HEATMAPS_LOSS: True
  WITH_OFFSETS_LOSS: True
MODEL:
  INIT_WEIGHTS: True
  NAME: 